In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from sklearn import metrics
import tabulate
import itertools
import umap
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import goatools
from goatools.anno.gaf_reader import GafReader
import Bio.UniProt.GOA as GOA
import gget

# locals
import utils as ut
reload(ut)

2023-01-06 15:48:27.264901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 15:48:27.377504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-01-06 15:48:27.377527: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-06 15:48:27.403647: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-06 15:48:28.2119

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [2]:
query = ['Col1a1', 'Cd36', 'C3', 'Cd81', 'Col1a2', 'Cd36', 'Lpl', 'Lrp1', 'Cav1', 'App', 'App', 'Cav1', 'Cadm1', 'Cadm1', 'Cdh5', 'Cdh5', 'H2', 'D1', 'Aplp2', 'Aplp2', 'H2', 'D1', 'Psap', 'Lrp1', 'C3', 'C3ar1', 'Apoe', 'Lrp1', 'C3', 'Lrp1', 'Col1a1', 'Cd93']
db = 'WikiPathways_2019_Mouse'
# db = 'KEGG_2019_Mouse'
# db = 'ontology'
ef = gget.enrichr(query, database=db)

pd.set_option('display.max_colwidth', 200)


ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

Fri Jan  6 15:48:29 2023 INFO Performing Enichr analysis using database WikiPathways_2019_Mouse.


,path_name,p_val,overlapping_genes
0,Alzheimers Disease WP2075,5.362282e-07,"[APP, LRP1, LPL, APOE]"
1,Statin Pathway WP1,5.877426e-07,"[LRP1, LPL, APOE]"
2,Dysregulated miRNA Targeting in Insulin/PI3K-AKT Signaling WP3855,2.454669e-04,"[COL1A1, COL1A2]"
3,Inflammatory Response Pathway WP458,3.278488e-04,"[COL1A1, COL1A2]"
4,Retinol metabolism WP1259,5.558029e-04,"[LPL, CD36]"
5,Focal Adhesion WP85,5.735958e-04,"[COL1A1, COL1A2, CAV1]"
6,Splicing factor NOVA regulated synaptic proteins WP1983,6.447805e-04,"[CADM1, APLP2]"
7,Complement and Coagulation Cascades WP449,1.401119e-03,"[C3, C3AR1]"
8,PPAR signaling pathway WP2316,2.376492e-03,"[LPL, CD36]"
9,ApoE and miR-146 in inflammation and atherosclerosis WP3592,7.178529e-03,[APOE]
